In [1]:
from probemapper.registration import ANTSPointTransformation, ANTSTransformationStages
from probemapper.io import save_nifti
import os, subprocess
import numpy as np
import pandas as pd

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220615_P202_laqueus_DiI_L-A29/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, f"indexing")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Transform channel coordinates

In [4]:
T = ANTSPointTransformation(
    ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin/",
    num_threads=8
)
T.set_transformations([
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_0GenericAffine.mat"), inverse=True),
    ANTSTransformationStages(transformation=os.path.join(rootdir, "analysis/ants/F2M_1InverseWarp.nii.gz"), inverse=False),
])
T.set_moving_points(os.path.join(rootdir, "analysis/skeleton/channel_skeleton.csv"))
T.set_outname(os.path.join(tmpdir, "channel_LUT.csv"))

In [5]:
T.run()

['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsApplyTransformsToPoints', '-d', '3', '-i', '/bucket/ReiterU/DBS/data/20220615_P202_laqueus_DiI_L-A29/analysis/skeleton/channel_skeleton.csv', '-o', '/flash/ReiterU/tmp/220620_P202_skeletonization/indexing/channel_LUT.csv', '-t', '[/bucket/ReiterU/DBS/data/20220615_P202_laqueus_DiI_L-A29/analysis/ants/F2M_0GenericAffine.mat,1]', '-t', '/bucket/ReiterU/DBS/data/20220615_P202_laqueus_DiI_L-A29/analysis/ants/F2M_1InverseWarp.nii.gz']



### Query region ID

In [6]:
from probemapper import skeletonization

In [7]:
df = pd.read_csv(os.path.join(tmpdir, "channel_LUT.csv"))
coords = np.array([df["Z"], df["Y"], df["X"]])

In [8]:
regions = skeletonization.query_region_id2(coords,
                     "/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                     100, coords.shape[1]-100, 10)

In [9]:
df["Region"] = regions

In [10]:
df.insert(0, "channel", df.pop("channel"))

In [11]:
display(df)

,channel,X,Y,Z,X_org,Y_org,Z_org,Region
0,-100,1625.97,2832.94,2224.28,1360.12,3187.220,2414.97,8
1,-99,1628.39,2828.07,2230.97,1362.48,3177.600,2416.97,8
2,-98,1630.85,2823.29,2237.62,1364.83,3167.980,2418.97,8
3,-97,1633.34,2818.65,2244.01,1367.19,3158.360,2420.98,8
4,-96,1635.84,2814.19,2250.30,1369.55,3148.740,2422.98,8
...,...,...,...,...,...,...,...,...
282,182,2422.26,1519.87,4242.61,1992.10,478.734,2870.82,1
283,183,2423.12,1512.92,4252.83,1991.75,469.050,2873.30,1
284,184,2423.84,1505.92,4263.03,1991.21,459.367,2875.72,1
285,185,2424.43,1498.71,4273.41,1990.49,449.684,2878.09,1


In [12]:
df.to_csv(os.path.join(tmpdir, "channel_LUT.csv"), index=False)

In [13]:
sk = skeletonization.generate_skeleton_image(coords[:, 100:],
                             "/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/Slicer_3D/Segmentation.seg.nrrd",
                             10)

In [14]:
sk = sk.astype("uint8")
save_nifti(sk, os.path.join(tmpdir, "probe_image.nii.gz"), 1, 1, 1)

In [15]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/220620_P202_skeletonization/indexing', 'deigo:/bucket/ReiterU/DBS/data/20220615_P202_laqueus_DiI_L-A29/analysis'], returncode=0)